# EBV DNA covariates

Consolidating various pieces of information from these dataframes:
- `ebv_equivalent_30x.csv`: the amount of EBV per person, normalized by 30x WGS coverage. See `01_Quantify_EBV_DNA`.
- `genomic_metrics.tsv`: includes `sex_at_birth` and `biosample_collection_date`.
- `demographic_df.csv`: includes `birth_datetime`. See `EBV_DNA_PheWAS/01_Query_PheWAS_inputs.ipynb`.

In [ ]:
# Load in libraries
suppressMessages(library(data.table))
suppressMessages(library(dplyr))
library(fastglm)
library(ggplot2)
library(BuenColors)
library(lubridate)
setwd('/home/jupyter/workspaces/ebvdnaphewas')

## EBV DNA quantification for everyone with WGS

In [ ]:
ebv_30x_df <- fread("../intermediate/ebv_equivalent_30x.csv")

To compare between masked and unmasked quantification, add the EBV quantification for all bases, including `wiped_here` positions:

In [ ]:
# this is a dataframe of EBV reads quantified per person (across all positions)
# mostly to get the IDs of people who have any EBV reads, before bias correction
ebv_DNA_unmasked <- fread("../data/ebv_reads_counts.tsv.gz")

There are a lot of people with reads at only `wipe_here` positions! Add this information in the `ebv_summary_df` in the section "Plot Classifications".

In [ ]:
table(ebv_DNA_unmasked$n_q30 > 0) # everyone with some q30 reads

In [ ]:
table(ebv_30x_df$ebv_q30_30x > 0.0018) # everyone with reads at non-wipe here positions + passed 0018 threshold

### Apply the same thresholds as in UKB

In [ ]:
thresholds <- c(0, 0.0015, 0.0018, 0.002, 0.003, 0.004, 0.005, 0.007, 0.012, 0.015, 0.03)

In [ ]:
counts <- vector(mode="integer", length=length(thresholds))
i = 1

for(thresh in thresholds){
    ebv_thresh <- ebv_30x_df %>% filter(ebv_q30_30x > thresh) 
    counts[i] <- nrow(ebv_thresh)
    i = i + 1
}

count.per.thresh <- data.frame(threshold = thresholds, EBV_count = counts)

### Convert thresholds to various units

In [ ]:
count.per.thresh %>%
dplyr::mutate(
threshold_1c = threshold / 15, # amount of EBV genomes per cell
threshold_cells = 1 / (threshold / 15), # amount of cells per EBV genome
threshold_10Kc = threshold / 15 * 10^4, # amount of EBV genomes per 10K cells
)

In [ ]:
ebv_30x_df$ebv_q30_1c <- ebv_30x_df$ebv_q30_30x / 15 # amount of EBV genomes per cell
ebv_30x_df$ebv_q30_cells <- 1 / ebv_30x_df$ebv_q30_1c # amount of cells per EBV genome
head(ebv_30x_df)

Plot cumulative frequency:

In [ ]:
# calculate cumulative frequency
freq_table <- as.data.frame(table(ebv_30x_df$ebv_q30_cells))
freq_table$cumulative_freq <- cumsum(freq_table$Freq)
total_count <- sum(freq_table$Freq)
freq_table$cumulative_freq_percent <- (freq_table$cumulative_freq / total_count) * 100
freq_table$cumulative_freq_percent_rev <- 100 - freq_table$cumulative_freq_percent
freq_table$ebv_q30_cells <- as.numeric(as.character(freq_table$Var1))
freq_table$ebv_q30_cells_log <- log10(freq_table$ebv_q30_cells)
p0 <- ggplot(freq_table, aes(x = ebv_q30_cells_log, y = cumulative_freq_percent_rev)) +
  geom_step() +
  labs(x = "Estimated # human cells per EBV genome (log10 scaled)",
       y = "Cumulative Frequency (%)") +
  scale_x_reverse() +
  pretty_plot(fontsize = 8) + L_border() 

Plot the top 100 people with the most EBV DNA load:

In [ ]:
ebv_30x_df <- ebv_30x_df %>% dplyr::arrange(desc(ebv_q30_30x)) %>% dplyr::mutate(rank_q30 = 1:n())
ebv_top100 <- ebv_30x_df[ebv_30x_df$rank_q30 <= 100,]

p1 <- ggplot(ebv_top100, aes(x = rank_q30, y = ebv_q30_cells)) + 
  geom_point(size = 0.5) +
  labs(x = "Top 100 AOU donors ranked",
       y = "# human cells per EBV genome") +
  scale_x_reverse() +
  scale_y_reverse() +
  pretty_plot(fontsize = 8) + L_border() 

### Annotate each individual based on whether they passed EBV thresholds

In [ ]:
for(thresh in thresholds){
    thresh_col <- paste0("ebv_q30_", thresh)
    ebv_30x_df[[thresh_col]] <- as.numeric(ebv_30x_df$ebv_q30_30x > thresh)
}
head(ebv_30x_df)

## Get covariates

In [ ]:
# get sex_at_birth 
gm <- data.frame(fread("../data/genomic_metrics.tsv")) %>% 
        rename(person = research_id)
gm <- gm %>% dplyr::select(person, sex_at_birth, biosample_collection_date)
gm[gm == ""] <- NA

# get date of birth 
demo_df <- fread("../data/demographic_df.csv") %>% dplyr::select(person_id, birth_datetime)

In [ ]:
gm <- gm %>% dplyr::left_join(demo_df, by = c("person" = "person_id"))

In [ ]:
# Get age at time of biosample collection
gm <- gm %>% dplyr::mutate(
        age = floor(interval(start = birth_datetime, end = biosample_collection_date)
                            / duration(num = 1, units = 'years')))

## Plot classifications

In [ ]:
# adding in total q30 per person, before wipe_here
ebv_DNA_unmasked <- fread("../data/ebv_reads_counts.tsv.gz")
ebv_DNA_unmasked <- ebv_DNA_unmasked %>%
dplyr::mutate(
    person = as.integer(gsub("ebv_", "", sample)), 
    has_unmasked_q30 = as.numeric(n_q30 > 0)) %>%
dplyr::select(person, has_unmasked_q30)

In [ ]:
ebv_30x_df <- ebv_30x_df %>% dplyr::left_join(ebv_DNA_unmasked, by = "person")

In [ ]:
ebv_df_summary <- ebv_30x_df %>%
  dplyr::select(person, ebv_q30_30x, has_unmasked_q30) %>%
  mutate(what = case_when(
    ebv_q30_30x > 0.0018 ~ "yes", # passed 0018 threshold 
    ebv_q30_30x > 0 ~ "valid", # passed 0 threshold but not 0018 threshold
    has_unmasked_q30 > 0 ~ "biased", # had q30 reads, but not after wipe-here
    TRUE ~ "anothing" # else
  )) %>%
  group_by(what) %>%
  summarize(count = n()) %>%
  mutate(perc = count / sum(count)*100)

p2 <- ggplot(ebv_df_summary, aes(x = what, y = perc)) +
  geom_bar(stat = "identity", fill = "lightgrey", color = "black") + 
  pretty_plot(fontsize = 8) + L_border() + 
  scale_y_continuous(expand = c(0,0))

In [ ]:
# check that numbers are correct
table(ebv_30x_df$has_unmasked_q30 == 0)
table(ebv_30x_df$has_unmasked_q30 > 0, ebv_30x_df$ebv_q30_30x > 0)
table(ebv_30x_df$ebv_q30_30x > 0, ebv_30x_df$ebv_q30_30x > 0.0018)

## Plot age \* sex_at_birth

In [ ]:
# merge gm with ebv_30x_df
ebv_df <- ebv_30x_df %>% 
dplyr::left_join(gm, by = "person")
ebv_df$sex_at_birth <- factor(ebv_df$sex_at_birth, levels = c("M", "F"))
ebv_df <- ebv_df %>%
dplyr::select(person, sex_at_birth, age, ebv_q30_0.0018) %>%
dplyr::rename(has_ebv = ebv_q30_0.0018)

There are 1900 people without `age` and 2852 people without `sex_at_birth`. Remove them from the analysis.

In [ ]:
ebv_df[ebv_df == ""] <- NA

In [ ]:
nrow(ebv_df[is.na(ebv_df$age)])

In [ ]:
min(ebv_df$age, na.rm = T)
max(ebv_df$age, na.rm = T)

In [ ]:
table(ebv_df$sex_at_birth)

In [ ]:
nrow(ebv_df[is.na(ebv_df$sex_at_birth)])

In [ ]:
ebv_df <- ebv_df %>% na.omit()
nrow(ebv_df)

In [ ]:
# make summary stat plots
# % EBV+ per age bucket (decades)
# Assign to age group
# min(eur_re$age) == 18, max(eur_re$age) == 88

year_interval <- 5

ebv_df <- ebv_df %>%
  mutate(age_group = cut(age, breaks = seq(0, 100, by = year_interval), 
                         labels = paste(seq(0, 100 - year_interval, by = year_interval), "-", seq(5, 100, by = year_interval)), 
                         right = FALSE))

In [ ]:
ebv_df %>% dplyr::group_by(sex_at_birth, age_group) %>% dplyr::summarize(count = n())

In [ ]:
ebv_df %>% dplyr::group_by(age_group) %>% dplyr::summarize(count = n())

In [ ]:
df_age_group <- ebv_df %>%
  # dplyr::filter(age >= 40) %>% # ages seem pretty evenly spread out, so not filtering here
  dplyr::group_by(age_group, sex_at_birth) %>%
  dplyr::summarise(ebv_positive_percent = mean(has_ebv == 1) * 100, 
                   var = var( (has_ebv == 1)), hits = sum(has_ebv == 1) ,
                   count = n()) %>% mutate(sem = (ebv_positive_percent)/sqrt(var*count))

In [ ]:
pagesex <- ggplot(df_age_group, aes(x = age_group, y = ebv_positive_percent, fill = as.factor(sex_at_birth))) +
  geom_bar(stat = "identity", color = "black", position = position_dodge()) +
  labs(x = "Age Group", y = "Percentage of EBV+ (%)") +
  theme_minimal() + 
  theme(axis.text.x = element_text(angle = 45, hjust = 1))  + 
  pretty_plot(fontsize = 8) + L_border() +
  geom_errorbar(aes( ymin=ebv_positive_percent-sem, ymax=ebv_positive_percent+sem), width=0.2, position = position_dodge(.9)) +
  scale_y_continuous(expand = c(0,0)) +
  scale_fill_manual(values = c("lightgrey", "darkgrey")) 

Get the p-values for each age group:

In [ ]:
lapply(unique(df_age_group$age_group), function(oag){
  ssdf <- df_age_group %>% filter(age_group == oag)
  data.frame(oag, p = (prop.test(x = ssdf$hits, n = ssdf$count))[["p.value"]])
}) %>% rbindlist()

## Subset covariates to only EUR individuals

This section generates the files that are inputs to the `02_Format_input_files.ipynb` notebook in the `EBV DNA GWAS` folder, specifically:
- `ebv_EUR_0018.csv`
- `all_covariates_EUR.csv`
* NOTE: for running EBV DNA as a quantitative trait, the `ebv_30x_df_EUR` dataframe was used.

To get covariates files for EUR individuals, refer to the ancestry file and descriptions here:
- https://support.researchallofus.org/hc/en-us/articles/4614687617556-How-the-All-of-Us-Genomic-data-are-organized-Archived-C2022Q4R13-CDRv7
- https://support.researchallofus.org/hc/en-us/articles/29475233432212-Controlled-CDR-Directory 
- Google bucket filepath: gs://fc-aou-datasets-controlled/v8/wgs/short_read/snpindel/aux/ancestry/ancestry_preds.tsv

In [ ]:
ancestry <- fread("../data/ancestry_preds.tsv")

In [ ]:
ancestry <- ancestry %>% dplyr::rename(person = person_id)

In [ ]:
ancestry_covar <- ancestry %>% dplyr::select(person, sex_at_birth, ancestry_pred, pca_features)

In [ ]:
ancestry_covar_EUR <- ancestry_covar %>% dplyr::filter(ancestry_pred == "eur")

In [ ]:
nrow(ancestry_covar_EUR) # 133578

In [ ]:
nrow(ancestry_covar_EUR) # 133578
nrow(ebv_30x_df) # 245394
nrow(gm) # 245394
length(intersect(ancestry_covar_EUR$person, ebv_30x_df$person)) # 133578
length(intersect(ancestry_covar_EUR$person, gm$person)) # 133578
length(intersect(intersect(ancestry_covar_EUR$person, gm$person), ebv_30x_df$person)) # 133578

In [ ]:
ebv_30x_df_EUR <- ebv_30x_df[ebv_30x_df$person %in% ancestry_covar_EUR$person,]
gm_EUR <- gm[gm$person %in% ancestry_covar_EUR$person,]

Write EBV DNA phenotype file for (1) all thresholds (2) only 0.0018: 

In [ ]:
fwrite(ebv_30x_df_EUR, file = "EBV_GWAS_data/EUR/ebv_30x_df_EUR_allthresh.csv", row.names = FALSE, sep = "\t", quote = FALSE)

In [ ]:
pheno_df <- ebv_30x_df_EUR %>% 
dplyr::select(person, ebv_q30_0.0018) %>%
dplyr::rename(has_ebv = ebv_q30_0.0018)

In [ ]:
fwrite(pheno_df, file = "EBV_GWAS_data/EUR/ebv_EUR_0018.csv", row.names = FALSE, sep = "\t", quote = FALSE)

Consolidate covariates file (for input to EBV GWAS):

In [ ]:
ancestry_covar_EUR <- ancestry_covar_EUR %>% dplyr::select(-sex_at_birth)

In [ ]:
gm_EUR <- gm_EUR %>% dplyr::left_join(ancestry_covar_EUR, by = c("person" = "person"))

In [ ]:
# has person, sex_at_birth, age, pca_features, and age2
gm_EUR <- gm_EUR %>%
dplyr::select(-biosample_collection_date, -birth_datetime, -ancestry_pred) %>%
dplyr::mutate(age2 = age^2)

Convert the `pca_features` values to lists, then to individual PC columns.

In [ ]:
library(jsonlite)
# convert pca_features column to lists 
gm_EUR$pca_features <- lapply(gm_EUR$pca_features, function(x) fromJSON(x))

# extract each PCA feature and create new columns in covariate_df
for (i in 0:14) {
  gm_EUR[[paste0("PC", i + 1)]] <- sapply(gm_EUR$pca_features, function(t) t[[i + 1]])
}

In [ ]:
gm_EUR <- gm_EUR %>% dplyr::select(-pca_features)

In [ ]:
fwrite(gm_EUR, file = "EBV_GWAS_data/EUR/all_covariates_EUR.csv", row.names = FALSE, sep = "\t", quote = FALSE)